# Write input json required for plotting intersection stats

```sh

## Create intersection file with sample, n_snps, n_indels
for i in $(ls vcfeval_outputs/*tp.vcf.gz); 
do
    sample=$(dirname $i | cut -d'/' -f1)
    n_snps=$(bcftools view -v snps $i | grep -vc "^#")
    n_indels=$(bcftools view -v indels $i | grep -vc "^#")
    echo -e "$sample\t$n_snps\t$n_indels" 
done > intersection_vcfeval_fav.tsv

## Create LRS-only file with sample, n_snps, n_indels
for i in $(ls vcfeval_outputs/*fp.vcf.gz); 
do
    sample=$(dirname $i | cut -d'/' -f1)
    n_snps=$(bcftools view -v snps $i | grep -vc "^#")
    n_indels=$(bcftools view -v indels $i | grep -vc "^#")
    echo -e "$sample\t$n_snps\t$n_indels" 
done > lrsonly_vcfeval_fav.tsv

## Create SRS-only file with sample, n_snps, n_indels
for i in $(ls vcfeval_outputs/*fn.vcf.gz); 
do
    sample=$(dirname $i | cut -d'/' -f1)
    n_snps=$(bcftools view -v snps $i | grep -vc "^#")
    n_indels=$(bcftools view -v indels $i | grep -vc "^#")
    echo -e "$sample\t$n_snps\t$n_indels" 
done > srsonly_vcfeval_fav.tsv
```

## Generate json - 
```py
python3 vcfeval-intersection-json.py \
    intersection_vcfeval_fav.tsv lrsonly_vcfeval_fav.tsv srsonly_vcfeval_fav.tsv vcfeval_fav_stats.json
```

# FAV intersection plot

In [ ]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mplpatches
import numpy as np
import json


################### CUSTOM STACKED BAR PLOT ###################

mpl.rcParams['pdf.fonttype'] = 42

iBlue = (88/255, 85/255, 120/255)
iGreen = (120/255, 172/255, 145/255)
iYellow = (248/255, 174/255, 51/255)
iPurple = (60/255, 62/255, 100/255)
iGreenDark = (81/255, 116/255, 95/255)
iRed = (192/255, 41/255, 46/255)
iOrange = (230/255, 87/255, 43/255)

figureWidth = 6
figureHeight = 6

plt.figure(figsize=(figureWidth, figureHeight))

panelWidth = 1.5
panelHeight = 4

## 1. Set all panels 
panel1 = plt.axes([0.8/figureWidth, 0.5/figureHeight, 3*panelWidth/figureWidth, 0.6*(panelHeight/figureHeight)])
panel2 = plt.axes([0.8/figureWidth, 3.0/figureHeight, 3*panelWidth/figureWidth, 0.6*(panelHeight/figureHeight)])
panel1.spines[['top', 'right']].set_visible(False)
panel2.spines[['bottom', 'right']].set_visible(False)

### import files
current_dir = os.getcwd()
with open(os.path.join(current_dir, "vcfeval_fav_stats.json")) as infile:
    stats_dict = json.load(infile)

## We need a stacked bar plot for each sample
ylims = [i for i in range(1, 23, 1)]
# Intersection, SRS-only, LRS-only
colors = ["#F6F5F1FF", "#D2848DFF", "#82A2BFFF"]

panels = [panel1, panel2]

for p, i in enumerate(['INDEL', 'SNP']):
    legend_elements = []
    samples = []
    index = 0
    for sample, stats in stats_dict.items():
        samples.append(sample)
        snp_stats = stats[i]
        move = 0
        col_code = 0
        for j in ['Intersection', 'SRS-only', 'LRS-only']:  # intersect type
            left = move
            bottom = ylims[index]
            width = snp_stats[j]
            height = (ylims[index + 1] - bottom)

            rectangle = mplpatches.Rectangle([left, bottom], width, height,
                                             facecolor=colors[col_code],
                                             edgecolor='black',
                                             label=j,
                                             linewidth=0.2  # width of the edge around rectangle
                                             )
            panels[p].add_patch(rectangle)
            if len(legend_elements) < 3:
                legend_elements.append(rectangle)

            # Add text annotation inside the bar
            x_text = left + width / 2  # Position the text at the center of the bar
            y_text = bottom + height / 2
            text = int(width) if width > 0 else ""  # Show the count if width is non-zero
            panels[p].text(x_text, y_text, text, ha='right', va='center', color='black', fontsize=6)

            move += snp_stats[j]
            col_code += 1
        index += 1

xmin = 0
xmax = 15001
ymin = 0
ymax = 23

for panel in [panel1, panel2]:
    panel2.set_xlim(xmin, xmax)
    panel1.set_xlim(0, 1200)
    panel.set_ylim(ymin, ymax)
    panel.set_yticks([i for i in np.arange(1.5, 22.5, 1)], [s for s in samples], fontsize=6)
    panel2.set_xticks([i for i in range(0, 15001, 2000)], [str(i) for i in range(0, 15001, 2000)], fontsize=8)
    panel1.set_xlabel('Number of INDELs', fontdict={'fontsize': 11})
    panel2.set_title('Number of SNPs', fontdict={'fontsize': 11})

panel1.tick_params(bottom=True, labelbottom=True,
                   left=True, labelleft=True,
                   right=False, labelright=False,
                   top=False, labeltop=False
                   )
panel2.tick_params(bottom=False, labelbottom=False,
                   left=True, labelleft=True,
                   right=False, labelright=False,
                   top=True, labeltop=True
                   )

plt.ylabel("Probands", fontdict={'fontsize': 12}, y=0)
plt.legend(handles=legend_elements, fontsize="8", ncol=1, loc='upper right', bbox_to_anchor=(1.03, 0.02))

plt.savefig('Figure2A.pdf', dpi=600)
